<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GS-for-NB" data-toc-modified-id="GS-for-NB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GS for NB</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-SVM" data-toc-modified-id="GS-for-SVM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GS for SVM</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-LR" data-toc-modified-id="GS-for-LR-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>GS for LR</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#RF" data-toc-modified-id="RF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>RF</a></span></li><li><span><a href="#VISUAL" data-toc-modified-id="VISUAL-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>VISUAL</a></span></li></ul></div>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Text-Classification/code")
!pip install pyLDAvis
!pip install gensim
!pip install pandas==1.3.0
import nltk
nltk.download('punkt')
nltk.download('stopwords')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [ ]:
nb_name = "Classification_svm_nb_baseline-upsampling+normalstring_stage2"

In [ ]:
import numpy as np 
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *
from data_utils import *


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
train, test = load_data()
train, upsampling_info = upsampling_train(train)

# train_text, train_label = train_augmentation(train, select_comb=[['text'], ['reply', 'reference_one']])
train_text, train_label = train_augmentation(train, select_comb=None)
test_text, test_label = test['text'], test['label']

test_text = test_text.apply(lambda x: normal_string(x))
train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']


In [ ]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label
y_test = test_label

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(11813,)
(7761,)
(11813,)
(7761,)


# GS for NB

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
              'clf__alpha': [0.6, 0.8, 1]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"NB_cv_result_{nb_name}.xlsx")

clf__alpha:	0.6
tfidf__binary:	True
tfidf__min_df:	3
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.892


## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# tfidf_vectorizer(train_text, test_text, min_df=2, max_df=100)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative 
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = MultinomialNB(alpha=0.6)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

num of words: 43214
X_train.shape (11813, 43214)
X_test.shape (7761, 43214)
classification_report:
f1:  {'micro avg': 0.8212859167633035, 'macro avg': 0.8051877553271476} 

                          precision  recall  f1-score  support
alt.atheism                   0.852   0.705     0.772    319.0
comp.graphics                 0.742   0.694     0.717    389.0
comp.os.ms-windows.misc       0.806   0.645     0.717    394.0
comp.sys.ibm.pc.hardware      0.641   0.819     0.719    392.0
comp.sys.mac.hardware         0.864   0.795     0.828    385.0
comp.windows.x                0.846   0.820     0.833    395.0
misc.forsale                  0.871   0.828     0.849    390.0
rec.autos                     0.922   0.901     0.912    395.0
rec.motorcycles               0.977   0.942     0.959    398.0
rec.sport.baseball            0.831   0.952     0.887    397.0
rec.sport.hockey              0.993   0.883     0.935    827.0
sci.crypt                     0.827   0.932     0.876    396.0
sci.elec

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,0.073046,0.022618,0.885516,0.982917,0.868991,0.982052,0.873114,0.982243,0.868991,0.982052
1,0.072318,0.020760,0.904527,0.980408,0.890620,0.979458,0.893831,0.979553,0.890620,0.979458
2,0.078019,0.020077,0.912235,0.979692,0.901795,0.978668,0.903803,0.978859,0.901795,0.978668
3,0.073167,0.020039,0.892529,0.980899,0.885879,0.979910,0.886670,0.980114,0.885879,0.979910


# GS for SVM

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
             'clf__penalty':['l2'], 'clf__C':[1,2,3]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"SVC_cv_result_{nb_name}.xlsx")

clf__C:	1
clf__penalty:	l2
tfidf__binary:	True
tfidf__min_df:	1
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.924


## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=1.0, dual=True, max_iter=3000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 127512
Dimension reduction with truncate SVD:
   input columns with  127512
   output columns with  1000


/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kwargs['lwork'] = ret[-2][0].real.astype(numpy.int)
/usr/local/lib/python3.7/dist-packages/scipy/linalg/decomp_qr.py:20: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, ch

X_train.shape (11813, 1000)
X_test.shape (7761, 1000)
classification_report:
f1:  {'micro avg': 0.8362324442726452, 'macro avg': 0.822356827110777} 

                          precision  recall  f1-score  support
alt.atheism                   0.806   0.718     0.760    319.0
comp.graphics                 0.737   0.748     0.742    389.0
comp.os.ms-windows.misc       0.766   0.764     0.765    394.0
comp.sys.ibm.pc.hardware      0.730   0.689     0.709    392.0
comp.sys.mac.hardware         0.799   0.847     0.822    385.0
comp.windows.x                0.857   0.805     0.830    395.0
misc.forsale                  0.836   0.900     0.867    390.0
rec.autos                     0.910   0.896     0.903    395.0
rec.motorcycles               0.952   0.947     0.950    398.0
rec.sport.baseball            0.795   0.955     0.867    397.0
rec.sport.hockey              0.988   0.878     0.930    827.0
sci.crypt                     0.904   0.932     0.918    396.0
sci.electronics               0

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,8.688616,0.019997,0.897117,0.974939,0.895058,0.974489,0.895584,0.974615,0.895058,0.974489
1,8.827355,0.018710,0.921051,0.971543,0.919743,0.971219,0.920078,0.971266,0.919743,0.971219
2,8.766235,0.018055,0.926775,0.970626,0.924822,0.970316,0.925571,0.970447,0.924822,0.970316
3,8.797470,0.018238,0.903547,0.972183,0.903827,0.972009,0.903489,0.972057,0.903827,0.972009


In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=1.0, dual=True, max_iter=3000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 127512
X_train.shape (11813, 127512)
X_test.shape (7761, 127512)
classification_report:
f1:  {'micro avg': 0.8581368380363354, 'macro avg': 0.8459959224306294} 

                          precision  recall  f1-score  support
alt.atheism                   0.847   0.746     0.793    319.0
comp.graphics                 0.760   0.815     0.787    389.0
comp.os.ms-windows.misc       0.814   0.779     0.796    394.0
comp.sys.ibm.pc.hardware      0.754   0.776     0.765    392.0
comp.sys.mac.hardware         0.831   0.870     0.850    385.0
comp.windows.x                0.887   0.818     0.851    395.0
misc.forsale                  0.858   0.915     0.886    390.0
rec.autos                     0.923   0.914     0.919    395.0
rec.motorcycles               0.960   0.957     0.958    398.0
rec.sport.baseball            0.822   0.965     0.888    397.0
rec.sport.hockey              0.992   0.897     0.942    827.0
sci.crypt                     0.930   0.937     0.933    396.0
sci.e

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,2.280226,0.020581,0.910841,0.999660,0.908937,0.999661,0.909340,0.999659,0.908937,0.999661
1,2.474596,0.018118,0.932224,0.999321,0.931256,0.999323,0.931610,0.999321,0.931256,0.999323
2,2.365592,0.018817,0.938437,0.999549,0.936675,0.999549,0.937557,0.999548,0.936675,0.999549
3,2.326915,0.019888,0.910766,0.999437,0.910261,0.999436,0.910265,0.999434,0.910261,0.999436


# GS for LR

In [15]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr', max_iter=1000))])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
             'clf__penalty':['l2'], 'clf__C':[1,2,3], 'clf__solver': ['liblinear', 'newton-cg', 'sag', 'saga' , 'lbfgs']}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"LR_1_cv_result_{nb_name}.xlsx")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated.

clf__C:	3
clf__penalty:	l2
clf__solver:	saga
tfidf__binary:	True
tfidf__min_df:	1
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.893


In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr'))])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
             'clf__penalty':['elasticnet'], 'clf__C':[1,2,3], 'clf__l1_ratio':[0.2, 0.5,0.8],'clf__solver': ['saga']}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"LR_2_cv_result_{nb_name}.xlsx")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_

## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LogisticRegression(penalty="l2", multi_class='ovr', max_iter=1000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

# RF

# VISUAL

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import precision_recall_fscore_support, classification_report, roc_curve, auc, precision_recall_curve
# mapping from integer feature name to original token string
feature_names = vectorizer.get_feature_names()

# Feature reduction with Kbest features based on chi2 score
ch2 = SelectKBest(chi2, k=50000)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)
if feature_names:
    # keep selected feature names
    feature_names = [feature_names[i] for i in ch2.get_support(indices=True)]

In [ ]:

p = (clf.predict_proba(test_dtm))[:, 1] 
p_binary = clf.predict(test_dtm)

p_label = 2
fpr, tpr, thresholds = roc_curve(test_y, p, pos_label=p_label)
auc_score = auc(fpr, tpr)

pre, rec, thresholds = precision_recall_curve(test_y, p, pos_label=p_label)

# calculate auc
prc_score = auc(rec, pre)

if show_plots:

    print(classification_report(test_y, p_binary))

    plt.subplot(1, 2, 1)

    plt.plot(fpr, tpr, color='darkorange', lw=2)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('AUC')

    plt.subplot(1, 2, 2)
    plt.plot(rec, pre, color='darkorange', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PRC')
    plt.tight_layout()

    plt.show()

print("AUC: {:.2%}, PRC: {:.2%}".format(auc_score, prc_score))
